In [1]:
import pandas as pd
import numpy as np

In [2]:
### Read csv file
df = pd.read_csv('./csv/nrw_hai_extracted.csv')

In [ ]:
df.nunique()

In [ ]:
len(df)

In [ ]:
df.isnull().sum().sum()

In [ ]:
# Function to calculate the centroid of the bounds
def calculate_centroid(bounds):
    lower_left_lon, lower_left_lat, upper_right_lon, upper_right_lat = bounds
    centroid_lon = (lower_left_lon + upper_right_lon) / 2
    centroid_lat = (lower_left_lat + upper_right_lat) / 2
    return centroid_lat, centroid_lon

In [ ]:
# Parse the BOUNDS field and calculate the centroid for each bound
df['BOUNDS'] = df['BOUNDS'].apply(eval)  # Convert the string representation of the list to an actual list

In [ ]:
df[['CENTROID_LAT', 'CENTROID_LON']] = df['BOUNDS'].apply(lambda b: pd.Series(calculate_centroid(b)))

In [ ]:
# Group by AGG_DAY_PERIOD and BOUNDS and compute the mean ACTIVITY_INDEX_TOTAL
grouped_df = df.groupby(['AGG_DAY_PERIOD', 'BOUNDS'], as_index=False).agg({
    'ACTIVITY_INDEX_TOTAL': 'mean'
}).rename(columns={'ACTIVITY_INDEX_TOTAL': 'MEAN_ACTIVITY_INDEX_TOTAL'})

In [ ]:
# Function to calculate the centroid of the bounds
def calculate_centroid(bounds):
    lower_left_lon, lower_left_lat, upper_right_lon, upper_right_lat = bounds
    centroid_lon = (lower_left_lon + upper_right_lon) / 2
    centroid_lat = (lower_left_lat + upper_right_lat) / 2
    return centroid_lat, centroid_lon

# Parse the BOUNDS field and calculate the centroid for each bound
extracted_df['BOUNDS'] = extracted_df['BOUNDS'].apply(eval)  # Convert the string representation of the list to an actual list
extracted_df[['CENTROID_LAT', 'CENTROID_LON']] = extracted_df['BOUNDS'].apply(lambda b: pd.Series(calculate_centroid(b)))

# Group by AGG_DAY_PERIOD and BOUNDS and compute the mean ACTIVITY_INDEX_TOTAL
grouped_df = extracted_df.groupby(['AGG_DAY_PERIOD', 'BOUNDS', 'CENTROID_LAT', 'CENTROID_LON'], as_index=False).agg({
    'ACTIVITY_INDEX_TOTAL': 'mean'
}).rename(columns={'ACTIVITY_INDEX_TOTAL': 'MEAN_ACTIVITY_INDEX_TOTAL'})

# Save the resulting DataFrame to a new CSV file
grouped_df.to_csv('mean_activity_index_per_day_bounds.csv', index=False)

print("The CSV file has been created successfully.")
